In [7]:
import pandas as pd
Data = pd.read_csv('C:/Users/muneeba/Desktop/THESIS/DataDistributionPCA.csv')
Data=Data.sort_values(['Sensor ID'],ascending=[True])
Data1=Data.iloc[:,4:68]
Data1.head()
returns = Data1[[key for key in dict(Data1.dtypes) if dict(Data1.dtypes)[key] in ['float64', 'int64']]] 
returns = returns[:64]
returns.head()
#slicing the required data out of complete dataset
returns_arr = returns.as_matrix()[:64]
returns_arr.shape
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
returns_arr_scaled = scaler.fit_transform(returns_arr)
#logdir = r'C:/Users/muneeba/Desktop/THESIS/logdir'

Set up the TensorFlow code for an Autoencoder
The number of inputs of the auto-encoder is equal to the number of outputs (number of columns in our input data)
We have one hidden layer with 3 neurons
Use an AdamOptimizer to minimize the loss
Notice that:
The reconstruction loss of the output is the mean-square error
Each layer has no activation function i.e is a linear layer
An Autoencoder set up this way simply performs a PCA

In [8]:
import tensorflow as tf

n_inputs = 64
n_hidden = 3  # codings
n_outputs = n_inputs

learning_rate = 0.01 #how much to train 
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = tf.layers.dense(X, n_hidden)
outputs = tf.layers.dense(hidden, n_outputs)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()




Run the training data through the Autoencoder
We do not use any labels for this training only the input data
The output is the reconstructed value using only the 3 principal components
Note that the reconstruction will not be equal to the input data

In [9]:
n_iterations = 10000

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: returns_arr_scaled})
    writer = tf.summary.FileWriter('./temp', sess.graph)
    outputs_val = outputs.eval(feed_dict={X: returns_arr_scaled})
    print(outputs_val)

[[ 0.97901708  0.99220878  1.02413929 ...,  0.81025791  0.62554532
   0.44420934]
 [ 0.67799622  0.68887252  0.7154392  ...,  0.75396359  0.60632855
   0.46138459]
 [-0.31315541 -0.3098537  -0.30077857 ...,  0.57233566  0.54665345
   0.52135414]
 ..., 
 [-0.21888119 -0.21534388 -0.19652672 ...,  0.16412792  0.22418936
   0.27175242]
 [ 0.98880297  1.00053835  1.05748701 ..., -0.50504923 -0.39012089
  -0.31223655]
 [ 1.20379519  1.21692455  1.2803874  ..., -0.62719482 -0.50280488
  -0.41969341]]


In [12]:
import numpy as np
data=scaler.inverse_transform(outputs_val,copy=None)
#recovered
#data
orig=np.linalg.norm(outputs_val,ord=1)
DiffData=data-returns_arr
NewNorm=np.linalg.norm(DiffData,ord=1)
error=NewNorm/orig 
error
#recovering the orignal dat

0.00012398916095726891

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Use the retry module or similar alternatives.


In [14]:
# import the data and split into X and Y

#test_data = np.array(pd.read_csv(r'data\fashion-mnist_test.csv'), dtype='float32')

embed_count = 1600
x_test = Data1[:embed_count, 1:] / 255
y_test = Data1[:embed_count, 0]

logdir = r'C:\Users\Fashion_MNIST\logdir'  # you will need to change this!!!

TypeError: unhashable type: 'slice'